In [ ]:
# Cell 1: Setup
%load_ext autoreload
%autoreload 2

import pandas as pd
from fund_lens_etl.clients.fec import FECAPIClient
from fund_lens_etl.extractors.fec import (
    FECScheduleAExtractor,
    FECCommitteeExtractor,
    FECCandidateExtractor,
)
from fund_lens_etl.loaders.bronze import (
    BronzeFECScheduleALoader,
    BronzeFECCommitteeLoader,
    BronzeFECCandidateLoader,
)
from fund_lens_etl.database import get_db_session
from fund_lens_models.enums import USState

print("✓ Imports successful")

In [ ]:
# Cell 2: Test Committee Extract + Load
print("="*60)
print("TEST 1: Extract & Load Committees")
print("="*60)

# Extract
client = FECAPIClient()
committee_extractor = FECCommitteeExtractor(api_client=client)
committee_df = committee_extractor.extract(committee_ids=["C00840017"])

print(f"Extracted {len(committee_df)} committees")
print(f"Columns: {committee_df.columns.tolist()}")

# Load
committee_loader = BronzeFECCommitteeLoader()
with get_db_session() as session:
    loaded = committee_loader.load(session, committee_df)
    print(f"\n✓ Loaded {loaded} committees to bronze")

In [ ]:
# Cell 3: Verify Committee in Database
print("="*60)
print("Verify Committee in Database")
print("="*60)

with get_db_session() as session:
    from fund_lens_etl.models.bronze.fec import BronzeFECCommittee
    from sqlalchemy import select

    stmt = select(BronzeFECCommittee).where(
        BronzeFECCommittee.committee_id == "C00840017"
    )
    committee = session.execute(stmt).scalar_one_or_none()

    if committee:
        print(f"✓ Found committee: {committee.name}")
        print(f"  Party: {committee.party_full}")
        print(f"  Type: {committee.committee_type_full}")
        print(f"  Filing frequency: {committee.filing_frequency}")
        print(f"  First file date: {committee.first_file_date}")
    else:
        print("✗ Committee not found!")

In [ ]:
# Cell 4: Test Candidate Extract + Load
print("="*60)
print("TEST 2: Extract & Load Candidates")
print("="*60)

candidate_extractor = FECCandidateExtractor(api_client=client)
candidate_df = candidate_extractor.extract(
    state=USState.MD,
    cycle=2026,
)

print(f"Extracted {len(candidate_df)} candidates")

# Load first 10 for testing
test_df = candidate_df.head(10)
candidate_loader = BronzeFECCandidateLoader()
with get_db_session() as session:
    loaded = candidate_loader.load(session, test_df)
    print(f"\n✓ Loaded {loaded} candidates to bronze")

In [ ]:
# Cell 5: Verify Candidates in Database
print("="*60)
print("Verify Candidates in Database")
print("="*60)

with get_db_session() as session:
    from fund_lens_etl.models.bronze.fec import BronzeFECCandidate

    stmt = select(BronzeFECCandidate).limit(5)
    candidates = session.execute(stmt).scalars().all()

    print(f"Found {len(candidates)} candidates in database")
    for candidate in candidates:
        print(f"\n  {candidate.name}")
        print(f"    Office: {candidate.office_full}")
        print(f"    Party: {candidate.party_full}")
        print(f"    District: {candidate.district}")
        print(f"    Has raised funds: {candidate.has_raised_funds}")